In [2]:
# We have a csv with the actual stop times.
# We have another CSV with the scheduled stop times.
# We want to combine them so we can run a linear regression.
# We use pandas

import pandas as pd 
CLEAN_DIRECTORY = 'C:/Users/nings/OneDrive - The University of Western Ontario/Scholar\'s 2200E/clean_data'
LTC_DIRECTORY = 'C:/Users/nings/Documents/GitHub/se-2200e/raw_data'


In [6]:
schedule_csv_path = LTC_DIRECTORY + '/schedule/stop_times.txt'
schedule_df = pd.read_csv(schedule_csv_path)
schedule_df = schedule_df.drop(columns=['stop_headsign', 'arrival_time', 'pickup_type', 'drop_off_type', 'timepoint'])
schedule_df.head()


,trip_id,departure_time,stop_id,stop_sequence
0,1342560,6:13:00,KIPPADEL,1
1,1342560,6:13:51,KIPPBELF,2
2,1342560,6:14:41,KIPPBARK,3
3,1342560,6:15:13,KIPPARBO,4
4,1342560,6:16:04,KIPPBRIA,5


In [13]:
actual_csv_path = CLEAN_DIRECTORY + '/result.csv'
actual_df = pd.read_csv(actual_csv_path, sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
actual_df.info()
actual_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771386 entries, 0 to 6771385
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   trip_id         object
 1   start_date      object
 2   start_time      object
 3   route_id        object
 4   stop_sequence   object
 5   departure_time  object
 6   stop_id         object
 7   vehicle_id      object
 8   vehicle_label   object
 9   timestamp       object
dtypes: object(10)
memory usage: 516.6+ MB


,trip_id,start_date,start_time,route_id,stop_sequence,departure_time,stop_id,vehicle_id,vehicle_label,timestamp
0,1346803,20201104,11:18:00,24,41,1604508849,WESTMAL1,3140,140,1604509170
1,1346803,20201104,11:18:00,24,42,1604508850,VISCFARN,3140,140,1604509170
2,1346803,20201104,11:18:00,24,43,1604508889,VISCVIGR,3140,140,1604509170
3,1346803,20201104,11:18:00,24,44,1604508928,VISCCRAN,3140,140,1604509170
4,1346803,20201104,11:18:00,24,45,1604508982,CRANBAR2,3140,140,1604509170


In [22]:
actual_df = actual_df.astype(dtype={
    'trip_id': 'int64',
    'route_id': 'int64',
    'stop_sequence': 'int64',
    'departure_time': 'datetime64',
    'vehicle_id': 'int64',
    'timestamp': 'datetime64'
}, errors='ignore')
actual_df['departure_time'] = pd.to_datetime(actual_df['departure_time'], unit='s')
actual_df['timestamp'] = pd.to_datetime(actual_df['timestamp'], unit='s')
actual_df.info()
actual_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771386 entries, 0 to 6771385
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   trip_id         int64         
 1   start_date      object        
 2   start_time      object        
 3   route_id        object        
 4   stop_sequence   int64         
 5   departure_time  datetime64[ns]
 6   stop_id         object        
 7   vehicle_id      object        
 8   vehicle_label   object        
 9   timestamp       datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 516.6+ MB


,trip_id,start_date,start_time,route_id,stop_sequence,departure_time,stop_id,vehicle_id,vehicle_label,timestamp
0,1346803,20201104,11:18:00,24,41,2020-11-04 16:54:09,WESTMAL1,3140,140,2020-11-04 16:59:30
1,1346803,20201104,11:18:00,24,42,2020-11-04 16:54:10,VISCFARN,3140,140,2020-11-04 16:59:30
2,1346803,20201104,11:18:00,24,43,2020-11-04 16:54:49,VISCVIGR,3140,140,2020-11-04 16:59:30
3,1346803,20201104,11:18:00,24,44,2020-11-04 16:55:28,VISCCRAN,3140,140,2020-11-04 16:59:30
4,1346803,20201104,11:18:00,24,45,2020-11-04 16:56:22,CRANBAR2,3140,140,2020-11-04 16:59:30


In [23]:
combined_df = actual_df.merge(schedule_df, how='inner', on=['trip_id', 'stop_id', 'stop_sequence'])
actual_df.info()
combined_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771386 entries, 0 to 6771385
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   trip_id         int64         
 1   start_date      object        
 2   start_time      object        
 3   route_id        object        
 4   stop_sequence   int64         
 5   departure_time  datetime64[ns]
 6   stop_id         object        
 7   vehicle_id      object        
 8   vehicle_label   object        
 9   timestamp       datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 516.6+ MB


,trip_id,start_date,start_time,route_id,stop_sequence,departure_time_x,stop_id,vehicle_id,vehicle_label,timestamp,departure_time_y
0,1346628,20201104,11:39:00,19,9,2020-11-04 16:47:57,SUNNYMCA,3141,141,2020-11-04 16:59:31,11:40:27
1,1346628,20201105,11:39:00,19,9,2020-11-05 16:43:38,SUNNYMCA,3317,317,2020-11-05 16:52:43,11:40:27
2,1346628,20201106,11:39:00,19,9,2020-11-06 16:43:52,SUNNYMCA,3108,108,2020-11-06 16:52:34,11:40:27
3,1346628,20201109,11:39:00,19,9,2020-11-09 16:42:28,SUNNYMCA,3153,153,2020-11-09 16:50:47,11:40:27
4,1346628,20201110,11:39:00,19,9,2020-11-10 16:42:13,SUNNYMCA,3303,303,2020-11-10 16:50:33,11:40:27
